In [1]:
import time
import json

from itertools import combinations
from typing import Dict
from mp_api.client import MPRester
from pymatgen.core import Element
from pymatgen.analysis.phase_diagram import PhaseDiagram
from pymatgen.entries.computed_entries import ComputedStructureEntry

Read the API key from a txt file

In [2]:
with open("mp-api-key.txt") as f:
    API_KEY = f.read()

# 1. Enumerate all the relevant chemical systems

In [3]:
elements_to_query = [
    "Al",
    "Na",
    "Li",
    "Co",
    "Zn", 
    "Cu",
    "Fe",
    "Si",
    "O",
    "S",
    "N"
]
atomic_numbers = [Element(sym).Z for sym in elements_to_query]
print(f"Atomic numbers: {atomic_numbers}")

Atomic numbers: [13, 11, 3, 27, 30, 29, 26, 14, 8, 16, 7]


In [4]:
# Create a mapping for one-hot encoding conversion
atomic_mapping = dict(zip(atomic_numbers, range(len(atomic_numbers))))
print(atomic_mapping)
# Save the mapping
with open("./data/mp-query/atomic_number_mapping.json", "w") as f:
    json.dump(atomic_mapping, f)

{13: 0, 11: 1, 3: 2, 27: 3, 30: 4, 29: 5, 26: 6, 14: 7, 8: 8, 16: 9, 7: 10}


In [5]:
# find all N choose 3 combinations
element_combos = list(combinations(elements_to_query, 3))
element_combos = sorted(["-".join(sorted(elems)) for elems in element_combos])
print(f"Total number of chemsys: {len(element_combos)}")
element_combos

Total number of chemsys: 165


['Al-Co-Cu',
 'Al-Co-Fe',
 'Al-Co-Li',
 'Al-Co-N',
 'Al-Co-Na',
 'Al-Co-O',
 'Al-Co-S',
 'Al-Co-Si',
 'Al-Co-Zn',
 'Al-Cu-Fe',
 'Al-Cu-Li',
 'Al-Cu-N',
 'Al-Cu-Na',
 'Al-Cu-O',
 'Al-Cu-S',
 'Al-Cu-Si',
 'Al-Cu-Zn',
 'Al-Fe-Li',
 'Al-Fe-N',
 'Al-Fe-Na',
 'Al-Fe-O',
 'Al-Fe-S',
 'Al-Fe-Si',
 'Al-Fe-Zn',
 'Al-Li-N',
 'Al-Li-Na',
 'Al-Li-O',
 'Al-Li-S',
 'Al-Li-Si',
 'Al-Li-Zn',
 'Al-N-Na',
 'Al-N-O',
 'Al-N-S',
 'Al-N-Si',
 'Al-N-Zn',
 'Al-Na-O',
 'Al-Na-S',
 'Al-Na-Si',
 'Al-Na-Zn',
 'Al-O-S',
 'Al-O-Si',
 'Al-O-Zn',
 'Al-S-Si',
 'Al-S-Zn',
 'Al-Si-Zn',
 'Co-Cu-Fe',
 'Co-Cu-Li',
 'Co-Cu-N',
 'Co-Cu-Na',
 'Co-Cu-O',
 'Co-Cu-S',
 'Co-Cu-Si',
 'Co-Cu-Zn',
 'Co-Fe-Li',
 'Co-Fe-N',
 'Co-Fe-Na',
 'Co-Fe-O',
 'Co-Fe-S',
 'Co-Fe-Si',
 'Co-Fe-Zn',
 'Co-Li-N',
 'Co-Li-Na',
 'Co-Li-O',
 'Co-Li-S',
 'Co-Li-Si',
 'Co-Li-Zn',
 'Co-N-Na',
 'Co-N-O',
 'Co-N-S',
 'Co-N-Si',
 'Co-N-Zn',
 'Co-Na-O',
 'Co-Na-S',
 'Co-Na-Si',
 'Co-Na-Zn',
 'Co-O-S',
 'Co-O-Si',
 'Co-O-Zn',
 'Co-S-Si',
 'Co-S-Zn',
 'Co-Si-Zn'

In [6]:
chemsys_with_errors = []
chemsys_to_screen = element_combos

# 2. Query the MP database

In [7]:
def process_entry(entry: ComputedStructureEntry,
                  pd: PhaseDiagram) -> Dict:
    """Process a ComputedStructureEntry queried from MP"""
    # Get the relevant ids
    mp_id = entry.data["material_id"]
    task_id = entry.data["task_id"]
    
    # Get the formation energy per atom
    eform_per_atom = pd.get_form_energy_per_atom(entry)
    
    # Get the primitive structure
    struct = entry.structure.get_primitive_structure()
    # Get the composition
    comp = struct.composition.as_dict()
    # Get the lattice cell
    cell = struct.lattice.matrix.tolist()
    # Get the atom positions in Cartesian coordinates
    positions = struct.cart_coords.tolist()
    # Get the atomic number of the species
    atomic_numbers = [species.Z for species in struct.species]
    
    return dict(
        mp_id=mp_id,
        task_id=task_id,
        comp=comp,
        positions=positions,
        atomic_numbers=atomic_numbers,
        cell=cell,
        eform_per_atom=eform_per_atom,
    )

Please make sure that you don't query MP too often. Otherwise you will get banned!

In [8]:
totnum_entries = 0
with MPRester(api_key=API_KEY) as mpr:
    for chemsys in chemsys_to_screen:
        print(f"Querying {chemsys}")
        thermotype = "R2SCAN"
        chemsys_entries = mpr.get_entries_in_chemsys(
            elements=chemsys,
            additional_criteria={"thermo_types": [thermotype]}
        )
        
        # Check if no entries has been returned
        num_entries = len(chemsys_entries)
        if num_entries == 0:
            chemsys_with_errors.append(chemsys)
            continue
        
        # Construct a PhaseDiagram object
        pd = PhaseDiagram(chemsys_entries)
        
        # Save all the ComputedStructureEntries
        entries_filename = f"{chemsys}_{thermotype}.json"
        entries_dir = "./data/mp-query/raw/"
        entries_savepath = entries_dir + entries_filename
        entries_records = {
            "chemsys": chemsys,
            "all_entries": [entry.as_dict() for entry in chemsys_entries]
        }
        with open(entries_savepath, "w") as f:
            json.dump(entries_records, f)
        
        # Process and save the entries to be read in by jraph
        totnum_entries += num_entries
        for entry in chemsys_entries:
            entry_dict = process_entry(entry, pd)
            entry_filename = f"{chemsys}_{entry_dict['task_id']}.json"
            entry_dir = f"./data/mp-query/processed/"
            entry_savepath = entry_dir + entry_filename
            with open(entry_savepath, "w") as f:
                json.dump(entry_dict, f)
        
        print("Waiting in between query")
        time.sleep(5)

Querying Al-Co-Cu


Retrieving ThermoDoc documents:   0%|          | 0/32 [00:00<?, ?it/s]

Waiting in between query
Querying Al-Co-Fe


Retrieving ThermoDoc documents:   0%|          | 0/34 [00:00<?, ?it/s]

Waiting in between query
Querying Al-Co-Li


...

Waiting in between query
Querying O-Si-Zn


Retrieving ThermoDoc documents:   0%|          | 0/178 [00:00<?, ?it/s]

Waiting in between query
Querying S-Si-Zn


Retrieving ThermoDoc documents:   0%|          | 0/107 [00:00<?, ?it/s]

Waiting in between query


In [9]:
print(f"Total number of entries: {totnum_entries}")

Total number of entries: 7930
